# Técnicas e algoritmos de ataques ao RSA

## Funções Auxiliares

In [157]:
from Crypto.Util.number import getPrime, long_to_bytes
from sympy import isprime
from typing import Callable
import time
from tqdm import tqdm
import random 
from math import gcd

with open('words_pt.txt', 'r', encoding='utf-8') as file:
    words = [line.strip() for line in file if line.strip()]

In [177]:
## Funções Aritmétricas
def isqrt(n):
    """
    Calcula a raiz quadrada inteira de n usando o método de Newton.
    
    Parâmetros:
    n (int): O número cujo valor inteiro da raiz quadrada será calculado.

    Retorna:
    int: A raiz quadrada inteira de n.
    """
    x = n
    y = (x + n // x) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

def is_perfect_square(n):
    '''
    If n is a perfect square it returns sqrt(n),
    
    otherwise returns -1
    '''
    h = n & 0xF; #last hexadecimal "digit"
    
    if h > 9:
        return -1 # return immediately in 6 cases out of 16.

    # Take advantage of Boolean short-circuit evaluation
    if (h != 2 and h != 3 and h != 5 and h != 6 and h != 7 and h != 8):
        # take square root if you must
        t = isqrt(n)
        if t*t == n:
            return t
        else:
            return -1
    
    return -1

def totiente(p: int, q: int):
    '''
    Função totiente em pq
    '''
    return (p-1)*(q-1)

def egcd(a: int, b: int): 
    '''
    Algoritmo Extendido de Euclides
    Retorna x, y, mdc(a,b) tal que ax + by = mdc(a,b)
    '''
    u, u1 = 1, 0
    v, v1 = 0, 1
    while b:
        q = a // b
        u, u1 = u1, u - q * u1
        v, v1 = v1, v - q * v1
        a, b = b, a - q * b
    return u, v, a

def mod_inverse(e: int, n: int):
    '''
    d tal que: de = 1 (mod n)
    Assumimos que é verdade que e é coprimo a n
    '''
    return egcd(e,n)[0]%n

def generate_e(p,q):
    phi_n = totiente(p,q)
    e = random.randint(2, phi_n - 1)
    while gcd(e, phi_n) != 1:
        e = random.randint(2, phi_n - 1)
    return e

def get_d_from_primes(e,p,q):
    return mod_inverse(e, totiente(p,q))

In [178]:
ch = 120
def GenerateKeys(small_d=None, small_e=None, close_primes=None, n_bits=1024):
    if close_primes == True:
        p = getPrime(15)
        q = getPrime(15)
        e = generate_e(p,q)
        d = get_d_from_primes(e,p,q)
    else:
        if small_e == True:
            p = getPrime(n_bits)
            q = getPrime(n_bits) 
            e = 3
            while gcd(e,(p-1)*(q-1))!= 1:
                p = getPrime(n_bits)
                q = getPrime(n_bits)\

            d = get_d_from_primes(e,p,q)

    
    N = p*q
    print("="*ch)
    print(f"Chave Privada Gerada: (p = {p}, q = {q}, d = {d})")
    print(f"Chave Pública Gerada: (N = {N}, e = {e})")
    print("="*ch)
    return p,q,d,N,e

def ApplyMethod(function: Callable[[int, int], tuple], N: int, e: int):
    start_time = time.time()  
    p,q,d = function(N,e)
    print("="*ch)
    print(f"Chave privada encontrada: (p = {p}, q = {q}, d = {d})")
    print(f"Tempo necessário: {time.time() - start_time} segundos.")
    print("="*ch)
    return p,q,d

def GenerateMessage(word_count):
    message = ' '.join(random.choice(words) for _ in range(word_count))
    print(f"Mensagem Enviada: {message}")
    print("="*ch)
    return message

    
def CriptMessage(message: str, N: int, e:int):
    max_block_size = (N.bit_length() - 1) // 8 

    message = message.encode('utf-8')
    blocks = [message[i:i + max_block_size] for i in range(0, len(message), max_block_size)]

    # Criptografa cada bloco individualmente
    ciphertext_blocks = []
    for block in blocks:
        int_ = int.from_bytes(block, byteorder='big')  # Converte o bloco para inteiro
        c = pow(int_, e, N)  # Criptografa o bloco
        ciphertext_blocks.append(c)
    return ciphertext_blocks

def DecryptMessage(ciphertext_blocks, d, n):
        """
        Descriptografa uma lista de blocos criptografados.
        
        Args:
            ciphertext_blocks (list): Lista de blocos criptografados como inteiros.
        
        Returns:
            str: Mensagem descriptografada como string.
        """

        # Descriptografa cada bloco
        plaintext_bytes = b""
        for block in ciphertext_blocks:
            plaintext_int = pow(block, d, n)  # Descriptografa o bloco
            block_bytes = plaintext_int.to_bytes((plaintext_int.bit_length() + 7) // 8, byteorder='big')
            plaintext_bytes += block_bytes  # Junta os bytes de cada bloco

        message = plaintext_bytes.decode('utf-8')  # Converte para string
        print(f"Mensagem Decifrada: {message}")
        print("="*ch)
        return message

def IntToMessage(self, ciphertext_blocks):
        """
        Descriptografa uma lista de blocos criptografados.
        
        Args:
            ciphertext_blocks (list): Lista de blocos criptografados como inteiros.
        
        Returns:
            str: Mensagem descriptografada como string.
        """
        d, n = self.private_key

        # Descriptografa cada bloco
        plaintext_bytes = b""
        for block in ciphertext_blocks:
            plaintext_int = pow(block, d, n)  # Descriptografa o bloco
            block_bytes = plaintext_int.to_bytes((plaintext_int.bit_length() + 7) // 8, byteorder='big')
            plaintext_bytes += block_bytes  # Junta os bytes de cada bloco

        return plaintext_bytes.decode('utf-8')  # Converte para string

###  Força Bruta

Realiza a fatoração de um número semi-primo usando a busca por seus divisores. 

In [129]:
def força_bruta(N, e):
    """
    Realiza fatoração de um número N que é igual à multiplicação de dois primos, 
    usando busca por divisores. 

    Este método funciona bem para números com fatores relativamente pequenos.

    Exemplo: 
        N = 1039 * 58484513 = 60765409007, e = 7.
    
    Retorna:
        (p, q, d): Dois fatores de N, onde p * q = N. 

    Caso N seja primo, retorna:
        (N, 1, d)
    """

    # Verifica se N é divisível por 2 (paridade)
    if N % 2 == 0:
        d = get_d_from_primes(e, 2, N // 2)
        return 2, N // 2, d

    # Define o limite da busca como a raiz quadrada de N
    limite = isqrt(N)

    # Itera apenas sobre números ímpares a partir de 3 até a raiz quadrada de N
    for t in range(3, limite + 1, 2):
        if N % t == 0:  # Verifica se t é um divisor de N
            d = get_d_from_primes(e, t, N // t)
            return t, N // t , d

    # Caso nenhum divisor seja encontrado, N é primo
    d = get_d_from_primes(e, N, 1)
    return N, 1 , d

In [205]:
p, q, d, N, e = GenerateKeys(small_e=True, n_bits=10)
message = GenerateMessage(10)
ciphertext_blocks = CriptMessage(message, N, e)

Chave Privada Gerada: (p = 659, q = 863, d = 378131)
Chave Pública Gerada: (N = 568717, e = 3)
Mensagem Enviada: maioria meu cada adeus pode da vossos nada aí com


In [206]:
p, q, d = ApplyMethod(força_bruta, N, e)
message = DecryptMessage(ciphertext_blocks, d, N)

Chave privada encontrada: (p = 659, q = 863, d = 378131)
Tempo necessário: 9.5367431640625e-05 segundos.
Mensagem Decifrada: maioria meu cada adeus pode da vossos nada aí com


# Método de Fatoração de Fermat

O método de fatoração de Fermat é baseado na ideia de que qualquer número \(n\) ímpar pode ser escrito como a diferença de dois quadrados:

$n = a^2 - b^2 = (a + b)(a - b).$

Com isso, o objetivo do método é encontrar \(a\) e \(b\) tais que a fatoração acima seja válida. 


Método de Fermat é mais eficiente quando os dois fatores \(p\) e \(q\) são próximos, ou seja, |p - q| é pequeno. \
Caso contrário, o algoritmo pode demorar muito, especialmente para números grandes como o fornecido.

In [207]:
def fermat_factorization(N, e):
    """
    Realiza a fatoração de Fermat em um número composto N.

    Assumimos que N é um produto de dois primos ímpares próximos, p e q,
    tal que N = p * q e p ≈ q.

    Parâmetros:
    N (int): O número a ser fatorado.
    e (int): O expoente público do RSA.

    Retorna:
    tuple: Os fatores primos p e q de N e a chave privada d.

    Lança:
    AssertionError: Se N não for igual ao produto dos fatores encontrados.
    """
    # Caso trivial: N é par
    if N % 2 == 0:
        p, q = 2, N // 2
        d = get_d_from_primes(e, p, q)  # Calcula d usando p e q
        return p, q, d
    
    # Inicia com x como a raiz quadrada inteira de N
    x = isqrt(N)

    # Verifica se x já é um divisor direto de N
    if N % x == 0:
        p, q = x, N // x
        d = get_d_from_primes(e, p, q)  # Calcula d usando p e q
        return p, q, d

    # Itera para encontrar a decomposição N = a² - b²
    for i in range(x + 1, 1 + (N + 1) // 2):
        val = i**2 - N  # Calcula b² = a² - N
        y = isqrt(val)  # Calcula y como a raiz quadrada de b²

        # Verifica se val é um quadrado perfeito
        if y**2 == val:
            p, q = i - y, i + y
            d = get_d_from_primes(e, p, q)  # Calcula d usando p e q
            return p, q, d  # Retorna os fatores (p, q) e a chave privada d

    # Se nenhum fator for encontrado, N é primo
    d = get_d_from_primes(e, N, 1)  # Calcula d para o caso de N ser primo
    return N, 1, d


In [208]:
p,q,d,N,e = GenerateKeys(close_primes=True)
mensagem_original = GenerateMessage(15) # Gera uma frase de 15 polavras
mensagem_criptografada = CriptMessage(mensagem_original, N, e)

Chave Privada Gerada: (p = 17029, q = 17891, d = 76181203)
Chave Pública Gerada: (N = 304665839, e = 43219747)
Mensagem Enviada: tão fazeis meses quem dezanove falta pouco exemplo naquela tem põem dizem ter neste foi


In [209]:
p,q,d = ApplyMethod(fermat_factorization, N, e)
message = DecryptMessage(mensagem_criptografada, d, N)

Chave privada encontrada: (p = 17029, q = 17891, d = 76181203)
Tempo necessário: 0.00015735626220703125 segundos.
Mensagem Decifrada: tão fazeis meses quem dezanove falta pouco exemplo naquela tem põem dizem ter neste foi


# Realizando o ataque com o expoente $e$ pequeno

In [210]:
from gmpy2 import iroot

def small_e_attack(c: int, e: int) -> int:
    """
    Realiza o ataque de expoente pequeno (small e attack) ao módulo RSA.
    Para um expoente pequeno e (e.g., e = 3),
    se a mensagem m elevada a e for menor que o módulo n, a cifra c pode ser
    revertida diretamente extraindo a raiz e-ésima de c.

    Parâmetros:
    c (int): O valor cifrado (ciphertext).
    e (int): O expoente público do RSA.

    Retorna:
    int: A mensagem decifrada.
    """
    return int(iroot(c, e)[0])

In [211]:
p,q,d,N,e = GenerateKeys(small_e=True)
mensagem_original = GenerateMessage(5) # Gera uma frase de 15 polavras
mensagem_original = int.from_bytes(mensagem_original.encode(), "big")
mensagem_criptografada = pow(mensagem_original,e, N)

Chave Privada Gerada: (p = 172993952601187172771554307789158806834447956509734125132631225540928327660247989061985715137681151694252035904240387401857283949743216926222964479510780859672904096749720416174160887981158681210315412646711890572771741655682245400226257495743605745277237456386257995962803415254096101404627764712973087013333, q = 100620322385884195582249804354226100671069654008246099898198746116403146794218843187656912136841483648558597047142164203759228721014064234533738503610338200940902237526146546288918451476824241284794149978223841580086038202840470000621068014926869437861809230066500300934356804291184728306142912361589714029809, d = 1160447152102654876443092677067660999731049464187183679930050489348133917267614251939532434600710700755626233417811245476985401563795594035567559976209685891574559482386985872998610601449944890131096739107711639704150406825098977370374124361138701944404752159818309036874870688303553808975367019840869350064948567568843939848369398200630336255

In [212]:
decrypted = small_e_attack(mensagem_criptografada, e)
decrypted = decrypted.to_bytes((decrypted.bit_length() + 7) // 8, "big")
print(f"Mensagem decifrada: {decrypted.decode()}")

Mensagem decifrada: final tiveram mais vós conselho


# Método de Pollard p-1

O algoritmo p−1 de Pollard é um algoritmo de fatoração de inteiros teórico de números, inventado por John Pollard em 1974. É um algoritmo de propósito especial, o que significa que é adequado apenas para inteiros com tipos específicos de fatores

1- Escolha de um número base: Comece escolhendo um número $a$, que é um número inteiro pequeno. Este número pode ser escolhido aleatoriamente.\
2- Escolha de um limite $B$: O algoritmo também exige um número limite $B$, que é um valor máximo de primos $p$ a serem usados. O limite $B$ será a "faixa" dentro da qual os divisores possíveis são procurados. O valor de $B$ afeta o tempo de execução do algoritmo: quanto maior $B$, mais tempo pode levar, mas também pode ser mais eficiente na descoberta de fatores pequenos.

escrever mais!

In [ ]:
import os
import sys
import math
from binascii import hexlify
from gmpy2 import (
    mpz, random_state, mpz_urandomb, next_prime, is_prime, lcm, powmod, fac
)

# Configuração de debug
_DEBUG = False

# Configuração da flag e do estado inicial do gerador aleatório
mensagem = "Método de Pollard's p-1!!!"
mensagem = mpz(hexlify(mensagem.encode()), 16)  # Convertendo a flag para um inteiro
SEED = mpz(hexlify(os.urandom(32)).decode(), 16)
STATE = random_state(SEED)

def get_prime(state, bits):
    """
    Gera um número primo com o número de bits especificado.

    Parâmetros:
    - state: Estado do gerador aleatório.
    - bits (int): Quantidade de bits do primo.

    Retorna:
    - mpz: Um número primo.
    """
    return next_prime(mpz_urandomb(state, bits) | (1 << (bits - 1)))

def get_smooth_prime(state, bits, smoothness=16):
    """
    Gera um primo "smooth", ou seja, um número primo que é próximo de
    um produto de fatores pequenos.

    Parâmetros:
    - state: Estado do gerador aleatório.
    - bits (int): Quantidade de bits desejada para o primo.
    - smoothness (int): Número de bits dos fatores pequenos.

    Retorna:
    - tuple: O primo gerado e seus fatores.
    """
    p = mpz(2)
    p_factors = [p]

    # Construir produto inicial de fatores pequenos
    while p.bit_length() < bits - 2 * smoothness:
        factor = get_prime(state, smoothness)
        p_factors.append(factor)
        p *= factor

    bitcnt = (bits - p.bit_length()) // 2

    # Ajustar até atingir o número desejado de bits
    while True:
        prime1 = get_prime(state, bitcnt)
        prime2 = get_prime(state, bitcnt)
        candidate = p * prime1 * prime2
        if candidate.bit_length() < bits:
            bitcnt += 1
            continue
        if candidate.bit_length() > bits:
            bitcnt -= 1
            continue
        if is_prime(candidate + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = candidate + 1
            break

    p_factors.sort()
    return p, p_factors

# Parâmetros RSA
e = 0x10001

# Gerar os primos p e q com fatores smooth
while True:
    p, p_factors = get_smooth_prime(STATE, 1024, 16)
    if len(p_factors) != len(set(p_factors)):
        continue

    q, q_factors = get_smooth_prime(STATE, 1024, 17)
    if len(q_factors) != len(set(q_factors)):
        continue

    factors = p_factors + q_factors
    if e not in factors:
        break

# Calcular os valores de RSA
n = int(p * q)
m = math.lcm(p - 1, q - 1)
d = pow(e, -1, m)
# Cifrar a mensagem
c = pow(int(mensagem), e, n)

In [ ]:
def pollards_p_minus_1(n, e, c, a=2, B=65535):
    """
    Realiza a fatoração de um número composto `n` usando o método de Pollard's p-1.
    Depois utiliza os fatores para decifrar uma mensagem RSA.

    Parâmetros:
    - n (int): O módulo RSA a ser fatorado.
    - e (int): O expoente público do RSA.
    - c (int): A mensagem cifrada (ciphertext).
    - a (int): A base inicial usada no método de Pollard. Default é 2.
    - B (int): O limite inicial para o cálculo de b! (fatorial). Default é 65535.

    Retorna:
    - str: A mensagem decifrada.
    """
    while True:
        # Calcula b! (fatorial) e tenta encontrar um divisor comum com n
        b = fac(B)
        tmp2 = pow(a, b, n) - 1  # a^(b!) mod n - 1
        mdc = gcd(tmp2, n)

        # Avalia o valor do gcd e ajusta o limite B, se necessário
        if mdc == 1:
            B += 1  # Aumenta o limite B para continuar
        elif mdc == n:
            B -= 1  # Diminui o limite B para ajustar
        else:
            # Encontramos um fator p
            p = mdc
            q = n // p
            assert p * q == n, "Erro na fatoração: p * q != n"

            # Calcula o expoente privado d
            d = get_d_from_primes(e,p,q)

            # Decifra a mensagem
            m = pow(c, d, n)
            return long_to_bytes(m).decode()

In [ ]:
message = pollards_p_minus_1(n, e, c)
print(f"Mensagem decifrada: {message}")

Mensagem decifrada: Método de Pollard's p-1!!!


# Wiener Attack

escrever aqui tbm...

In [ ]:
def miller_rabin_pass(a, s, d, n):
	''' 
	n is an odd number with
		n-1 = (2^s)d, and d odd
		and a is the base: 1 < a < n-1
	
	returns True iff n passes the MillerRabinTest for a 
	'''
	a_to_power = pow(a, d, n)
	i=0
	#Invariant: a_to_power = a^(d*2^i) mod n
	
	# we test whether (a^d) = 1 mod n
	if a_to_power == 1:
		return True
	
	# we test whether a^(d*2^i) = n-1 mod n
	# 	for 0<=i<=s-1
	while(i < s-1):
		if a_to_power == n - 1:
			return True
		a_to_power = (a_to_power * a_to_power) % n
		i+=1
	
	# we reach here if the test failed until i=s-2	
	return a_to_power == n - 1

def miller_rabin(n):
	'''
	Applies the MillerRabin Test to n (odd)
	
	returns True iff n passes the MillerRabinTest for
	K random bases
	'''
	#Compute s and d such that n-1 = (2^s)d, with d odd
	d = n-1
	s = 0
	while d%2 == 0:
		d >>= 1
		s+=1
	
	#Applies the test K times
	#The probability of a false positive is less than (1/4)^K
	K = 20
	
	i=1
	while(i<=K):
	# 1 < a < n-1
		a = random.randrange(2,n-1)
		if not miller_rabin_pass(a, s, d, n):
			return False
		i += 1

	return True

def gen_prime(nbits):
	'''
	Generates a prime of n bits using the
	miller_rabin_test
	'''
	while True:
			p = random.getrandbits(nbits)
			#force p to have nbits and be odd
			p |= 2**nbits | 1
			if miller_rabin(p):
				return p
				break

def gen_prime_range(start, stop):
	'''
	Generates a prime within the given range
	using the miller_rabin_test
	'''
	while True:
		p = random.randrange(start,stop-1)
		p |= 1
		if miller_rabin(p):
				return p
				break

def generateKeys(nbits=512):
    '''
    Generates a key pair
        public = (e,n)
        private = d 
    such that
        n is n bits long
        (e,n) is vulnerable to the Wiener Continued Fraction Attack
    '''
    #nbits >= 1024 is recommended
    assert nbits%4==0
    
    p = gen_prime(nbits)
    q = gen_prime_range(p+1, 2*p)
    n = p*q
    phi = totiente(p, q)
        
    # generate a d such that:
    #     (d,n) = 1
    #    36d^4 < n
    good_d = False
    while not good_d:
        d = random.getrandbits(nbits//4)
        if (gcd(d,phi) == 1 and 36*pow(d,4) < n):
            good_d = True
                    
    e = modInverse(d,phi)
    return p,q, e,n,d

In [ ]:
def rational_to_contfrac(x: int, y: int) -> tuple[list[int], list[tuple[int, int]]]:
    """
    Converts a rational x/y fraction into a list of partial coefficients [a0, ..., an], and
    a list of convergents at each coefficient level [(n0, d0), (n1, d1), ...]

    O algortimo está disponível na seção 9.1 de https://r-knott.surrey.ac.uk/Fibonacci/cfINTRO.html#CFtofract

    Parâmetros:
        x (int): numerator of the given rational number
        y (int): denominator of the given rational number

    Retorna:
        tupla: a tuple of coefficients and convergents at each
        coefficient level
    """
    a = x // y
    cflist = [a]
    cvlist = [(a, 1)]
    ppn, ppd = 1, 0  # pre-pre numerator and denominator of convergent
    pn, pd = a, 1  # pre numerator and denominator of convergent
    while a * y != x:
        x, y = y, x - a * y
        a = x // y
        cflist.append(a)
        cn, cd = a * pn + ppn, a * pd + ppd
        cvlist.append((cn, cd))
        ppn, ppd = pn, pd
        pn, pd = cn, cd
    return cflist, cvlist


def wiener_attack(e: int, n: int) -> int:
    '''
    Finds d knowing (e,n)
    applying the Wiener continued fraction attack
    '''
    _, convergents = rational_to_contfrac(e, n)
    
    for (k,d) in convergents:
        #check if d is actually the key
        if k!=0 and (e*d-1)%k == 0:
            phi = (e*d-1)//k
            s = n - phi + 1
            # check if the equation x^2 - s*x + n = 0 has integer roots
            discr = s*s - 4*n
            if(discr>=0):
                t = is_perfect_square(discr)
                if t!=-1 and (s+t)%2==0:
                    return d

In [ ]:
p,q,e,n,d = generateKeys()

In [ ]:
wiener_attack(e,n)

92594152413712729618284307022639983197

# Interface 

In [ ]:
import time
from math import gcd, isqrt
import emoji


class SimuladorAtaqueRSA:
    def __init__(self, n, e=None, c=None, m=None, d=None):
        self.n = n
        self.e = e
        self.c = c
        self.m = m
        self.d = d

    def fatoracao_fermat(self):
        print(f"\n🔍 Executando Fatoração de Fermat para N={self.n}")
        p, q, d = fermat_factorization(self.n, self.e)
        print(f"🎯 Os fatores primos de {self.n} são:\n p = {p}\n q = {q}")
        print(f"🔑 A chave privada é: d = {d}")
        message = DecryptMessage(self.c, d, self.n)
        print(f"🔓 Mensagem decifrada: {message}")

        return
       

    def pollard_p_menos_1(self, B=10**6):
        # print(f"\n⚡ Executando Método Pollard p-1")
        message = pollards_p_minus_1(n, e, c)
        print(f"Mensagem decifrada: {message}")
        

def menu_principal():
    print("""
  ██████╗ ███████╗ █████╗     █████╗ ████████╗████████╗ █████╗  ██████╗██╗  ██╗
  ██╔═██║ ██╔════╝██╔══██╗   ██╔══██╗╚══██╔══╝╚══██╔══╝██╔══██╗██╔════╝██║ ██║
  ██║ ██║ ██║     ██║  ██╗   ██║  ██╗   ██║      ██║   ██║  ██╗██║     ██║ ██║
  ████║   ███████╗███████║   ███████║   ██║      ██║   ███████║██║     ████║
  ██║ ██╗      ██║██╔══██║   ██╔══██║   ██║      ██║   ██╔══██║██║     ██╔═██║
  ██║  ██║███████║██║  ██║   ██║  ██║   ██║      ██║   ██║  ██║╚██████╗██║  ██║
  ╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝   ╚═╝  ╚═╝   ╚═╝      ╚═╝   ╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝

                                               .----.
                                .------------. | == |
                                |.-========-.| |----|
                                ||          || | == |
                                ||          || |----|
                                |'-..... ..-'| |::::|
                                  `"")---(""`  |___.|
                                /::::::::::::\"__  "
                               /::::=======:::\  \`\ 
              <!--    __________   __  ______  ______       -->
              <!--   / __/ ___| | / / / __/  |/  / _ | ___  -->
              <!--  / _// (_ /| |/ / / _// /|_/ / __ |/ _ \ -->
              <!-- /_/  \___/ |___/ /___/_/  /_/_/ |_/ .__/ -->
              <!--                                  /_/     -->  

12537508673247452740415065632494689029027732950928380086256250024867187926128380
80287907219466080126910572928733538110840068106336709993591705426997197501264265
37797307222843941099150274102793206236042838704774026115105372038660088467920614
59271374450331751970488423409043692985323238050322405992071593773700020204905132
60850861669417224690840035708894072063281961794947745531844449231173387949796417
48472790731929346064345414201887091548869282344794136434276044068819968354034453
81296531655214965307063040023445866227631412406489130525362013420587466334224983
906533723001807034527677883322082180058876687814462664149 %¨$$#¨&*(()((¨#@$&_)))
               

""")

    print("🤖 Bem-vindo ao Simulador de Ataques RSA!")
    escolha = int(input("""Escolha o tipo de ataque:\n
          [1] Fatoração de Fermat \n
          [2] Método de Pollard p-1\n
                        
          Sua escolha: """))
    



    t_inicio = time.perf_counter()
    if escolha == 1:
        print("🔑 Por favor, insira os dados necessários para o ataque:")
        c = int(input("🔐 Digite a mensagem para realizar o ataque: "))
        n = int(input("🔢 Digite o número a ser fatorado,, chave pública (N) : "))
        e = int(input("🔑 Digite o valor de expoente da chave pública (e): ") )
        print("✅ Dados recebidos com sucesso!", n , e)
        ataque = SimuladorAtaqueRSA(n,e,c)
        resultado = ataque.fatoracao_fermat()
    elif escolha == 2:
        print(" Por favor, insira os dados necessários para o ataque:")
        n = int(input("🔑 Digite o valor de n (chave pública): ") )
        e = int(input("🔑 Digite o valor de e (chave pública): ") )
        c = int(input("🔐 Digite o valor de c (valor cifrado): "))
        print("✅ Dados recebidos com sucesso!")
        ataque = SimuladorAtaqueRSA(n,e,c)
        mensagens = [
        "[*] Hackeando sistemas...",
        "[*] Acesso autorizado!",
        "[*] Preparando ataque...",
        "[*] RSA Attack: Ativado!",
        "[*] Ataque completo!"
        ]
            
        time.sleep(1)

        print(f"\n⚡ Executando o Método Pollard p-1")

        for msg in mensagens:
            print(msg)
            time.sleep(2)  # Pausa entre as mensagens
        resultado = ataque.pollard_p_menos_1()

    else:
        print("⚠️ Escolha inválida.")
        return
    t_fim = time.perf_counter()

    # print("\n🎯 Resultado do ataque:")
    # print(f"{resultado}")
    print(f"⏱️ Tempo de execução: {t_fim - t_inicio:.2f} segundos")

if __name__ == "__main__":
    menu_principal()


  ██████╗ ███████╗ █████╗     █████╗ ████████╗████████╗ █████╗  ██████╗██╗  ██╗
  ██╔═██║ ██╔════╝██╔══██╗   ██╔══██╗╚══██╔══╝╚══██╔══╝██╔══██╗██╔════╝██║ ██║
  ██║ ██║ ██║     ██║  ██╗   ██║  ██╗   ██║      ██║   ██║  ██╗██║     ██║ ██║
  ████║   ███████╗███████║   ███████║   ██║      ██║   ███████║██║     ████║
  ██║ ██╗      ██║██╔══██║   ██╔══██║   ██║      ██║   ██╔══██║██║     ██╔═██║
  ██║  ██║███████║██║  ██║   ██║  ██║   ██║      ██║   ██║  ██║╚██████╗██║  ██║
  ╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝   ╚═╝  ╚═╝   ╚═╝      ╚═╝   ╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝

                                               .----.
                                .------------. | == |
                                |.-========-.| |----|
                                ||          || | == |
                                ||          || |----|
                                |'-..... ..-'| |::::|
                                  `"")---(""`  |___.|
                                /::::::::::::"__  "
              

KeyboardInterrupt: Interrupted by user

# Referências:

- https://braghetto.eti.br/files/trabalho%20oficial%20final%20rsa.pdf
- https://www.iacr.org/archive/crypto2007/46220388/46220388.pdf